In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import warnings
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:
model = "result_rtmdet_0226"
confidence = "01"
result_file = model+"_"+str(confidence)+".csv"

In [ ]:
df_eval = pd.read_csv(result_file)

In [ ]:
df_eval[df_eval['eval'] == "TP"]

## Optimize the confidence

In [ ]:
df_eval[(df_eval['gt']=='impression') & (df_eval['eval']=="TP")].sort_values("confidence").head()

In [ ]:
df_eval[(df_eval['gt']=='einriss') & (df_eval['eval']=="TP")]['confidence'].hist()

In [ ]:
df_eval[df_eval.filename.str.contains('235117')]

In [ ]:
# Assuming your dataframe is named df

# Find rows that are duplicates based on 'gt', 'gt_location', and 'filename'
duplicates = df_eval.duplicated(subset=['gt', 'gt_location', 'filename'], keep=False)

# Separate the dataframe into rows that are duplicates and those that are not
df_duplicates = df_eval[duplicates]
df_non_duplicates = df_eval[~duplicates]

# Drop rows where 'pred' is null in the duplicate rows
df_duplicates = df_duplicates.dropna(subset=['pred'])

# Combine the non-duplicate rows and the cleaned duplicate rows
df_eval = pd.concat([df_non_duplicates, df_duplicates])

In [ ]:
df_eval[df_eval['filename'].str.contains("_915")]

In [ ]:
# Define classes
df_eval_matrix = df_eval.copy()
classes = ["impression", "einriss", "abriss", "asperity", "ausseinriss"]

# Replace null values for 'gt' and 'pred' with 'None'
df_eval_matrix['gt'] = df_eval_matrix['gt'].fillna('None')
df_eval_matrix['pred'] = df_eval_matrix['pred'].fillna('None')

# Ensure all labels in gt and pred are among the defined classes or 'None'
valid_labels = set(classes + ['None'])
# assert set(df_eval_matrix['gt']).issubset(valid_labels), "Unexpected values in 'gt'"
# assert set(df_eval_matrix['pred']).issubset(valid_labels), "Unexpected values in 'pred'"

# Calculate the confusion matrix
cm = confusion_matrix(df_eval_matrix['gt'], df_eval_matrix['pred'], labels=classes + ['None'])

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes + ['None'])
disp.plot(cmap="viridis", xticks_rotation="vertical")
disp.ax_.set_title("Confusion Matrix")
disp.ax_.set_xlabel("Predicted Label")
disp.ax_.set_ylabel("Ground Truth")
plt.show()

In [ ]:
# Confusion matrix in image level
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import os
# Specify the directory
directory = 'split//batch_1//images'

# Get the list of file names
file_names = os.listdir(directory)

# Filter out directories, if needed
full_list = [f.split(".")[0] for f in file_names if os.path.isfile(os.path.join(directory, f))]

# Example data
ground_truth_rejected = df_eval[df_eval['gt'].notnull()]['filename'].drop_duplicates().to_list()
predicted_rejected = df_eval[df_eval['pred'].notnull()]['filename'].drop_duplicates().to_list()

# Calculate true positives, false positives, true negatives, and false negatives
tp = len(set(ground_truth_rejected) & set(predicted_rejected))
fn = len(set(ground_truth_rejected) - set(predicted_rejected))
fp = len(set(predicted_rejected) - set(ground_truth_rejected))
tn = len(set(full_list) - set(ground_truth_rejected) - set(predicted_rejected))

# Construct confusion matrix
cm = np.array([[tp, fn],
               [fp, tn]])

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Rejected', 'Accepted'])
disp.plot(cmap='viridis')

plt.title('Confusion Matrix')
plt.show()


In [ ]:
from pathlib import Path
def crop_image(original_image):
    offset = 20
    gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    threshold_value = np.mean(gray) + np.std(gray)
    _, thresholded_image = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY)

    contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("No contours found!")
        return

    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    x_crop = max(0, x - offset)
    y_crop = max(0, y - offset)
    w = min(original_image.shape[1] - x_crop, w + 2 * offset)
    h = min(original_image.shape[0] - y_crop, h + 2 * offset)
    cropped_image = original_image[y_crop:y_crop+h, x_crop:x_crop+w]
    
    return cropped_image 
    
def show_image_pairs(image_pairs, mode, def_name):
    """
    Display multiple pairs of images side by side.
    
    :param image_pairs: List of tuples, where each tuple contains paths to two images (image1_path, image2_path).
    """
    
    for idx, (filename, img2_path, gt, pred) in enumerate(image_pairs):
        # Read images
        fig_each, axes_each = plt.subplots(nrows=1, ncols=2, figsize=(20, 2))
        fig_each.suptitle(f"{def_name} - {mode}", fontsize=16)
        
        for file_name in os.listdir("split\\batch_1\\" + model +"\\" +str(confidence)+ "\\misdetections\\"):
            if filename in file_name:  # Check if part_of_name is in file_name
                img1_path = os.path.join("split\\batch_1\\" + model + "\\" +str(confidence)+ "\\misdetections\\", file_name)
            else:
                pass
#                 img1_path = img2_path

        if not os.path.exists(img1_path):
            print(f"Error: File not found -> {img1_path}")
        if not os.path.exists(img2_path):
            print(f"Error: File not found -> {img2_path}")
        img1 = cv2.cvtColor(cv2.imread(img1_path), cv2.COLOR_BGR2RGB)
        img1 = crop_image(img1)
        img2 = cv2.cvtColor(cv2.imread(img2_path), cv2.COLOR_BGR2RGB)
        
        if mode == "TP":
            img1 = cv2.cvtColor(cv2.imread(img2_path), cv2.COLOR_BGR2RGB)
        
        # Display the first image
        axes_each[0].set_title(f"{filename} {mode} {pred}")
        axes_each[0].imshow(img1)
        
        # Display the second image
        axes_each[1].set_title(f"Output")
        axes_each[1].imshow(img2)
        if not os.path.exists("split\\batch_1\\" + model + "\\" +str(confidence)+ "\\error_pairs"):
            os.makedirs("split\\batch_1\\" + model + "\\" +str(confidence)+ "\\error_pairs")
        
        fig_each.savefig("split\\batch_1\\" + model + "\\" +str(confidence)+ "\\error_pairs\\"+mode+"_"+def_name+"_"+Path(img2_path).name.split(".")[0]+'.jpg')
    
        plt.tight_layout(rect=[0, 0, 1, 0.98])
        plt.show()


In [ ]:
def_names =  [ "einriss", "abriss", "ausseinriss", "impression", "asperity" ]
# def_names =  [ "abriss"]
type = "FN" # FN, FP
# modes = ["merge", "wrong", "notdetect"]
modes = ["notdetect"]

for def_name in def_names:
    
    for mode in modes:

        if mode == "merge":
            fil_df_eval = df_eval[(df_eval['gt']==def_name) & (df_eval['eval']==type) & (df_eval['pred'] == df_eval['gt'])]
        elif mode == "wrong":
            fil_df_eval = df_eval[(df_eval['gt']==def_name) & (df_eval['eval']==type) & (df_eval['pred'] != df_eval['gt']) & df_eval['pred'].notnull()]
        elif mode == "notdetect":
            fil_df_eval = df_eval[(df_eval['gt']==def_name) & (df_eval['eval']==type) & (df_eval['pred'].isnull())]
            
        if fil_df_eval.shape[0] != 0:
            # fil_df_eval['gt_path'] = "split\\batch_1\\" + model + "\\" + str(confidence) +  "\\images\\" + fil_df_eval['filename'] + ".bmp"
            fil_df_eval['gt_path'] = "split\\batch_1\\" + model + "\\" + str(confidence) +  "\\image_unfilter_crop\\" + fil_df_eval['filename'] + ".bmp"
            # fil_df_eval['gt_path'] = "split\\batch_1\\images_crop\\"+ fil_df_eval['filename'] + ".jpg"
            fil_df_eval = fil_df_eval.sort_values("pred")
            image_pairs = list(zip(fil_df_eval['filename'], fil_df_eval['gt_path'], fil_df_eval['gt'], fil_df_eval['pred']))

            show_image_pairs(image_pairs, mode, def_name)

In [ ]:
# def_names =  [ "einriss", "abriss", "ausseinriss", "impression", "asperity" ]
def_names = ['abriss']
type = "FP" # FN, FP
# modes = ["wrong", "redundant"]
modes = [ "redundant"]

for def_name in def_names:
    for mode in modes:

        if mode == "wrong":
            fil_df_eval = df_eval[(df_eval['pred']==def_name) & (df_eval['eval']==type) & (df_eval['pred'] != df_eval['gt']) & df_eval['gt'].notnull()]
        elif mode == "redundant":
            fil_df_eval = df_eval[(df_eval['pred']==def_name) & (df_eval['eval']==type) & (df_eval['gt'].isnull())]
            
        if fil_df_eval.shape[0] != 0:
            fil_df_eval['gt_path'] = "split\\batch_1\\"+ model +"\\"+ confidence +"\\image_unfilter_crop\\" + fil_df_eval['filename'] + ".bmp"
            fil_df_eval = fil_df_eval.sort_values("gt")
            image_pairs = list(zip(fil_df_eval['filename'], fil_df_eval['gt_path'], fil_df_eval['pred'], fil_df_eval['gt']))

            show_image_pairs(image_pairs, mode, def_name)

In [ ]:
# def_names =  [ "einriss", "abriss", "ausseinriss", "impression", "asperity" ]
def_names = ['abriss']
type = "TP"
modes = ["TP"]

for def_name in def_names:
    for mode in modes:

        fil_df_eval = df_eval[(df_eval['pred']==def_name) & (df_eval['eval']==type)]
            
        if fil_df_eval.shape[0] != 0:
            fil_df_eval['gt_path'] = "split\\batch_1\\"+ model +"\\"+ confidence +"\\images_unfilter_crop\\" + fil_df_eval['filename'] + ".bmp"
            fil_df_eval = fil_df_eval.sort_values("gt")
            image_pairs = list(zip(fil_df_eval['filename'], fil_df_eval['gt_path'], fil_df_eval['pred'], fil_df_eval['gt']))

            show_image_pairs(image_pairs, mode, def_name)